In [2]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari
from datetime import datetime

In [3]:
# path to wherever the s2p-lbm repository is cloned on your computer
repo_path = os.path.dirname(os.path.abspath(""))
os.chdir(repo_path)

from suite3d.job import Job
from suite3d import lbmio, utils, ui
from suite3d import tiff_utils as tfu

In [4]:
results_dir = '/mnt/md0/s3d-benchmarking/results/reg'

In [10]:
job = Job('/mnt/md0/s3d-benchmarking','benchmark-reg',create=False, verbosity = 3)
# job.copy_init_pass( Job('/mnt/md0/s3d-benchmarking','benchmark-init', create=False).load_summary())

   Found and loaded params from /mnt/md0/s3d-benchmarking/s3d-benchmark-reg/params.npy


In [18]:
tic = time.time()
job.register_gpu()
toc = time.time()

      Found dir /mnt/md0/s3d-benchmarking/s3d-benchmark-reg/registered_fused_data with tag registered_fused_data
Will analyze 1 tifs in 1 batches
   Enforcing positivity
   Launching IO thread
         Memory at batch 0.  Total Used: 244.185 GB, Virtual Available: 274.697 GB, Virtual Used: 228.499 GB, Swap Used: 015.685 GB
Loading Batch 0 of 0
      Loading /mnt/md0/data/demo/AH007_ROI3p33um_NOSCREEN_-400um_00001_00001.tif


Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid st

   Loaded 1 files, total 6.82 GB
   Batch 0 IO thread joined
         Memory after IO thread joinTotal Used: 250.985 GB, Virtual Available: 267.896 GB, Virtual Used: 235.300 GB, Swap Used: 015.685 GB
         Memory after movie copied from threadTotal Used: 257.816 GB, Virtual Available: 261.065 GB, Virtual Used: 242.131 GB, Swap Used: 015.685 GB
         Memory after thread memory clearedTotal Used: 250.991 GB, Virtual Available: 267.890 GB, Virtual Used: 235.306 GB, Swap Used: 015.685 GB
      Loaded batch of size (30, 100, 1202, 1015)
      Sending frames 0-10 to GPU for rigid registration
         Loaded mov and masks to GPU for rigid reg in 0.50 sec
         Subtracting min pix vals to enforce positivity
         Subtracting crosstalk
         Fusing and padding movie
         GPU Mov of shape 30, 10, 1239, 991; 2.74 GB
         Allocating memory for shifted movie
         Registered batch in 0.02 sec
         Shifted batch in 0.30 sec
         Freeing all blocks
      Completed r

         Loaded mov and masks to GPU for rigid reg in 0.50 sec
         Subtracting min pix vals to enforce positivity
         Subtracting crosstalk
         Fusing and padding movie
         GPU Mov of shape 30, 10, 1239, 991; 2.74 GB
         Allocating memory for shifted movie
         Registered batch in 0.02 sec
         Shifted batch in 0.30 sec
         Freeing all blocks
      Completed rigid registration in 1.21 sec
         Allocated GPU array for non-rigid reg in 0.19 sec
   Clipped movie in 0.00 sec
   Split movie into blocks in 0.00 sec
   Completed FFT of blocks and computed phase correlations in 0.00 sec
   Computed SNR and smoothed phase corrs in 0.35 sec
         Computed subpixel shifts in 0.01 sec
      Computed non-rigid shifts in 1.76 sec
      Transferred shifted mov of shape (10, 30, 1239, 991) to CPU in 0.00 sec
   Non rigid transformed (on CPU) in 2.49 sec
         After GPU Batch:    Total Used: 263.143 GB, Virtual Available: 255.739 GB, Virtual Used: 247.457

In [19]:
rigid_y, rigid_x, nonrigid_x, nonrigid_y = job.load_registration_results()

In [23]:
mov = job.get_registered_movie('registered_fused_data', 'fused')

In [25]:
mean_vol = mov.mean(axis=1).compute()

In [12]:
offset = job.get_registered_files(filename_filter='offset')

offset = n.load(offset[0],allow_pickle=True).item()

In [13]:
offset.keys()

dict_keys(['xmaxs_rr', 'ymaxs_rr', 'xmaxs_nr', 'ymaxs_nr'])

In [14]:
offset['xmaxs_rr'].shape

(100, 30)

In [15]:
offset['xmaxs_nr'].shape

(100, 30, 180)

In [34]:
timings = {'register'    : toc - tic}
outputs = {'rigid_y'     : rigid_y,
           'rigid_x'     : rigid_x,
           'nonrigid_y'  : nonrigid_y,
           'nonrigid_x'  : nonrigid_x,
           'mean_vol'    : mean_vol}
repo_status = utils.get_repo_status(repo_path)

In [36]:
# this updates the baseline
# utils.save_benchmark_results(results_dir, outputs, timings, repo_status, is_baseline=True)

In [37]:
utils.benchmark(results_dir, outputs, timings, repo_status)

                                         Baseline        |      Current         | 
Branch:                             dev                  | dev                  | 
Last commit hash:                   30c58f16ab717fa9dc43 | 30c58f16ab717fa9dc43 | 
Last commit summ:                   benchmarking/testing | benchmarking/testing | 
Dirty :                             True                 | True                 | 
    
Timings (s) 
register                                          70.124 |               70.124 | 

Outputs: 
rigid_y                                                  |                      |  mismatch: 0 / 3000 (100.00000 % match) 
           shape:                   (100, 30)            | (100, 30)            | 
           mean:                                 -22.053 |              -22.053 |
           std:                                   14.442 |               14.442 | 
rigid_x                                                  |                      |  mismatch: 0 / 3000 